In [96]:
import numpy as np
import sys
from rdkit import Chem
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
import rdkit
from rdkit import Chem
import numpy
from rdkit.Chem.Descriptors import MolWt
import pandas as pd
import scipy.sparse as sp



def preprocess_features(features):
    rowsum = np.array(features.sum(axis=1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return rowsum



def AddEdges(atom, n):
    #The Adjacentacy matrix is the bonds from the atoms
    Adj1=np.zeros((n,n))
    Adj2=np.zeros((n,n))
    Adj3=np.zeros((n,n))
    Adj4=np.zeros((n,n))
    for i, mol in enumerate(atom):
        first=mol.GetBeginAtomIdx()
        second = mol.GetEndAtomIdx()
        bond_type = mol.GetBondType()
        if bond_type==rdkit.Chem.rdchem.BondType.SINGLE:
            Adj1[first][second]=1
        elif bond_type==rdkit.Chem.rdchem.BondType.DOUBLE:
            Adj2[first][second]=1
        elif bond_type==rdkit.Chem.rdchem.BondType.TRIPLE:
            Adj3[first][second]=1
        else:
            Adj4[first][second]=1
    
    Adj=Adj1+Adj2+Adj3+Adj4
    return(Adj)

def Dia_M(adj):
    #Diagonal Matrix, rowwise summation of Adju matrix
    dia_len=len(adj)
    Diagonal_M=np.zeros((dia_len, dia_len))
    Dia_sum=adj.sum(axis=1)
    for i in range (0, dia_len):
        Diagonal_M[i][i]=Dia_sum[i]
    return Diagonal_M

def normalize_adj(adj, diagonal_m):
    #Applying self-loop and performing D^-1/2 * A * D^1/2 
    n=len(adj)
    I=np.identity(n)#Identity matrix 
    adj = np.asmatrix(I+adj)
    rowsum = np.array(diagonal_m.sum(axis=1)) #sum of the diagonal matrix rowwise
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0
    #print(d_inv_sqrt)
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt) 
    TempA=d_inv_sqrt @ d_mat_inv_sqrt.transpose()
    final_adj=TempA @ adj
    return final_adj

def convertToGraph(Molecule):
    Mol = Chem.MolFromSmiles(Molecule)
    Mol=Chem.AddHs(Mol)
    maxNumAtoms=Mol.GetNumAtoms()
    Adju_matrix=AddEdges(Mol.GetBonds(), maxNumAtoms)
    Diagonal_matrix=Dia_M(Adju_matrix)
    iFeatureTmp = []
    for atom in Mol.GetAtoms():
        iFeatureTmp.append( atom_feature(atom) ) 
    features=np.matrix(iFeatureTmp)
    SubFinal_compu=normalize_adj(Adju_matrix, Diagonal_matrix)
    #print(Adju_matrix)
    print(SubFinal_compu)
    return features, Adju_matrix, SubFinal_compu
    
def atom_feature(atom):
    return np.array(Atom_symbol(atom.GetSymbol(),
                                      ['C', 'N', 'O', 'S', 'F', 'H', 'Si', 'P', 'Cl', 'Br',
                                       'Li', 'Na', 'K', 'Mg', 'Ca', 'Fe', 'As', 'Al', 'I', 'B',
                                       'V', 'Tl', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn',
                                       'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'Mn', 'Cr', 'Pt', 'Hg', 'Pb']) +
                    [atom.GetDegree()] +
                    [atom.GetTotalNumHs()] + 
                    [atom.GetIsAromatic()])   

def Atom_symbol(x, Atom_set):
    
    atom_list=[]
    if x not in Atom_set:
        x = Atom_set[-1]
    for i in Atom_set:
        if i==x:
            atom_list.append(1)
        else:
            atom_list.append(0)
    return atom_list

def Graph_features(smiles_list, k):
    for i in smiles_list:
        Feature_Matrix, Adju_matrix, Adj_final_matrix=convertToGraph(i.strip())
        #print(Feature_Matrix)

        
      



    
    
train_set=pd.read_csv("/Users/tarakeswariramachandra/Documents/Michigan State University/Volunteer project /GCN-papers/gcn-task/data_train.csv")
train_label=train_set['label']
smiles_f=np.savetxt('/Users/tarakeswariramachandra/Documents/Michigan State University/Volunteer project /GCN-papers/gcn-task/smiles_string.txt', train_set.smiles, fmt='%s')
smiles_f = open('/Users/tarakeswariramachandra/Documents/Michigan State University/Volunteer project /GCN-papers/gcn-task/smiles_string.txt')
smiles_list = smiles_f.readlines()
length=len(train_set)
Graph_features(smiles_list[0:length], 1)

    

/Users/tarakeswariramachandra/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in power


[[0.25       0.58333333 0.83333333 2.         1.5        1.
  1.         1.         1.         1.         1.5        0.5
  1.83333333 1.5        1.         0.83333333 1.         2.
  1.5        1.5        1.5        1.         1.         0.5
  1.83333333 1.33333333 0.66666667 1.33333333 1.33333333 0.66666667
  0.25       0.25       0.25       0.33333333 0.5        0.5
  0.5        0.5        0.5        1.         0.5        0.33333333
  0.33333333 0.33333333 0.5        0.5        0.33333333 0.33333333
  0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333]]
[[0.25       0.58333333 0.83333333 2.         2.         1.5
  0.5        1.5        2.5        1.5        1.         1.
  1.         1.         1.83333333 1.5        1.         1.
  1.         1.         0.25       0.25       0.25       0.33333333
  0.5        0.5        0.5        0.5        0.5        0.5
  0.5        0.5        0.5        0.5       ]]
[[0.25       0.58333333 0.83333333 0.83333333 1.83333333 1.5
  1.